In [19]:
from glob import glob
from tqdm import tqdm
import json

In [29]:
files = glob('*-00000-of-00001.jsonl')
files

['cpp-00000-of-00001.jsonl',
 'scala-00000-of-00001.jsonl',
 'c-sharp-00000-of-00001.jsonl',
 'shell-00000-of-00001.jsonl',
 'go-00000-of-00001.jsonl',
 'python-00000-of-00001.jsonl',
 'pythonanalytics-00000-of-00001.jsonl',
 'sql-00000-of-00001.jsonl',
 'java-00000-of-00001.jsonl',
 'dockerfile-00000-of-00001.jsonl',
 'rust-00000-of-00001.jsonl',
 'javascript-00000-of-00001.jsonl',
 'typescript-00000-of-00001.jsonl',
 'python-analytics-00000-of-00001.jsonl',
 'csharp-00000-of-00001.jsonl',
 'ruby-00000-of-00001.jsonl',
 'php-00000-of-00001.jsonl',
 'swift-00000-of-00001.jsonl',
 'cuda-00000-of-00001.jsonl',
 'kotlin-00000-of-00001.jsonl']

In [30]:
filtered = []
rejected = []

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            if 'code snippet' in l['instruction']:
                rejected.append(l)
                continue
            answer = l['answer'].split('\n\n')
            if len(answer[0]) < 20:
                answer = answer[1:]
            answer = '\n\n'.join(answer)
            if len(answer.split()) < 100:
                continue
            filtered.append((l['instruction'], answer))
        print(f, len(filtered))
len(filtered)

10747it [00:00, 64703.93it/s]


cpp-00000-of-00001.jsonl 8119


10017it [00:00, 54691.22it/s]


scala-00000-of-00001.jsonl 16832


5693it [00:00, 67367.21it/s]


...

13843it [00:00, 66295.75it/s]


cuda-00000-of-00001.jsonl 211002


12790it [00:00, 63449.72it/s]

kotlin-00000-of-00001.jsonl 219018


219018

In [31]:
rejected[-1]

{'instruction': 'In the realm of culinary arts, an essential aspect is the ability to manage and share recipes. In this problem, you will be working with a simplified Recipe Data Access Object (DAO) that manages a list of recipes. The `Recipe` class encapsulates recipe details, while the `RecipeDao` class provides methods for inserting, updating, deleting, and fetching recipes by ID, name, or index.\n\nThe provided code snippet contains tests that showcase how the `RecipeDao` class functions. However, the implementation of these methods is missing. Your task is to implement the missing methods based on the provided tests and requirements.\n\nThe `Recipe` class:\n\n```kotlin\nclass Recipe(\n    val id: Int,\n    val recipe_name: String,\n    val author: String,\n    val description: String,\n    val ingredients: MutableList<String>,\n    val last_modified_date: Date,\n    val rating: Int\n)\n```\n\nThe `RecipeDao` class with missing methods:\n\n```kotlin\nclass RecipeDao {\n\n    privat

In [34]:
filtered[10000]

('In this problem, you are given a custom `PackedDate` class representing a date as bitfields of an integer. This class is designed to avoid unnecessary allocations and allow for fast access to day, month, and year fields as well as sorting. However, it is not suitable for complex date arithmetic operations.\n\nYour task is to implement a function `daysUntilYearEnd(packedDate: PackedDate): Int` that calculates the remaining days until the end of the year for the given `PackedDate` object.\n\nHere is the `PackedDate` class:\n\n```\nclass PackedDate(val value: Int) {\n  def day: Int = value & 0x1f\n  def month: Int = (value >> 5) & 0xf\n  def year: Int = (value >> 9) & 0x3fff\n\n  def asDays: Int = {\n    // Converts the packed date to the total days since a reference date\n    // You may use this method to make calculations easier\n  }\n}\n\nobject PackedDate {\n  def apply(value: Int): PackedDate = new PackedDate(value)\n\n  def apply(year: Int, month: Int, day: Int) =\n    new PackedD

In [35]:
with open('postfilter.jsonl', 'w') as fopen:
    for l in filtered:
        d = {
            'instruction': l[0],
            'answer': l[1]
        }
        fopen.write(f'{json.dumps(d)}\n')

In [38]:
from huggingface_hub import HfApi
api = HfApi()

In [39]:
api.upload_file(
    path_or_fileobj='postfilter.jsonl',
    path_in_repo='postfilter.jsonl',
    repo_id='mesolitica/mixtral-wizardcoder',
    repo_type='dataset',
)

postfilter.jsonl:   0%|          | 0.00/709M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/mixtral-wizardcoder/blob/main/postfilter.jsonl'